In [1]:
# Load symbolicregression model

import json
import torch
import os, sys
import symbolicregression
import sympytorch
import requests
from sympy.core.rules import Transform

model_path = "ckpt/model.pt" 
try:
    if not os.path.isfile(model_path): 
        print("Downloading model...")
        url = "https://dl.fbaipublicfiles.com/symbolicregression/model1.pt"
        r = requests.get(url, allow_redirects=True)
        open(model_path, 'wb').write(r.content)
    if not torch.cuda.is_available():
        sr_model = torch.load(model_path, map_location=torch.device('cpu'))
    else:
        sr_model = torch.load(model_path)
        sr_model = sr_model.cuda()
    print(sr_model.device)
    print("Model successfully loaded!")

except Exception as e:
    print("ERROR: model not loaded! path was: {}".format(model_path))
    print(e)    
    
est = symbolicregression.model.SymbolicTransformerRegressor(
                        model=sr_model,
                        max_input_points=10001,
                        n_trees_to_refine=5,
                        rescale=True
                        )

cuda:0
Model successfully loaded!


In [2]:
import sympy as sp
from sympy import symbols, exp, sin, Wild, sympify
import sympy as sp
from sympy.core.function import AppliedUndef, Function
from utils.utils import *


def get_first_function_name(expr):
    # Function to recursively search for the first function in the expression
    if isinstance(expr, Function) and not isinstance(expr, AppliedUndef):
        return expr.func.__name__
    elif expr.args:
        for arg in expr.args:
            name = get_first_function_name(arg)
            if name:
                return name
    return None

# Function to check if expression matches the pattern
def check_pattern(expr, pattern):    
    # Attempt to match the expression to the pattern
    match_result = expr.match(pattern)
    
    # Check if the pattern matched
    if match_result:
        for v, c in match_result.items():
            if not c.is_constant():
                return False, match_result
        return True, match_result
    else:
        return False, None

def extract_nonpoly_coefficients(expr, function):
    t = symbols('t')
    # Define wildcards for the pattern
    a1, a2, a3, a4, a5 = Wild('a1'), Wild('a2'), Wild('a3'), Wild('a4'), Wild('a5')
    wildcards = [a1, a2, a3, a4, a5]
    default_values = {a1:1., a2:1., a3:0., a4:0., a5:0.}
    # Define the pattern
    patterns = [a1*function(a2*t + a3) + a4*t + a5,
                a1*function(a2*t + a3) + a5]
    matched = False
    match_result = None
    for pattern in patterns:
        matches, match_result = check_pattern(expr, pattern)
        print(matches)
        print(match_result)
        if matches:
            matched = True
            break
    if not matched:
        return None
    result = []
    for wc in wildcards:
        if wc in match_result:
            result.append(float(match_result[wc]))
        else:
            result.append(default_values[wc])
    return result

x = symbols('x')
# Example expression that follows the pattern
expr = sympify("1.5*sin(2.0*x+2)-2.0*x+3")
display(expr)

a1, a2, a3, a4, a5 = Wild('a1'), Wild('a2'), Wild('a3'), Wild('a4'), Wild('a5')
pattern = a4*x + a1*sin(a2*x + a3) + a5
            
matches, match_result = check_pattern(expr, pattern)
print(matches)
print(match_result)

# Check the pattern
coeffs = extract_nonpoly_coefficients(expr, sin)
print(coeffs)

expr = sp.sympify("4.4*sin(t)")
#print(get_first_function_name(f))
coeffs = extract_nonpoly_coefficients(expr, sin)
print(coeffs)
assert coeffs == [4.4, 1.0, 0.0, 0.0, 0.0]

f = sp.simplify(sp.sympify('-3.0*sin(4.0*t)'))
print(get_first_function_name(f))
result = extract_nonpoly_coefficients(f, sin)
print(result)
assert result == [-3.0, 4.0, 0.0, 0.0, 0.0]

f = sp.simplify(sp.sympify('-3.0*sin(4.0*t+2.3)'))
print(get_first_function_name(f))
result = extract_nonpoly_coefficients(f, sin)
print(result)
assert result == [-3.0, 4.0, 2.3, 0.0, 0.0]
    
f = sp.simplify(sp.sympify("3-1.5*sin(2.9*t+2)+2.0*t"))
print(get_first_function_name(f))
result = extract_nonpoly_coefficients(f, sin)
print(result)
assert result == [-1.5, 2.9000000000000, 2.0, 2.0, 3.0]

f = sp.sympify("3-1.5*sin(t+2)-2.0*t")
print(get_first_function_name(f))
result = extract_nonpoly_coefficients(f, sin)
print(result)
assert result == [-1.5, 1.0, 2.0, -2.0, 3.0]

f = sp.simplify(sp.sympify("exp(t)-2.0*t"))
print(get_first_function_name(f))
result = extract_nonpoly_coefficients(f, exp)
print(result)
assert result == [1.0, 1.0, 0.0, -2.0, 0.0]

f = sp.simplify(sp.sympify("sin(2.3*t)-3"))
print(get_first_function_name(f))
result = extract_nonpoly_coefficients(f, sin)
print(result)
assert result == [1.0, 2.3, 0.0, 0.0, -3.0]

f = sp.simplify(sp.sympify("sin(t)"))
print(get_first_function_name(f))
result = extract_nonpoly_coefficients(f, sin)
print(result)
assert result == [1.0, 1.0, 0.0, 0.0, 0.0]

f = sp.simplify(sp.sympify("cos(2*t)"))
print(get_first_function_name(f))
result = extract_nonpoly_coefficients(f, cos)
print(result)
assert result == [1.0, 2.0, 0.0, 0.0, 0.0]

f = sp.simplify(sp.sympify("2*t*exp(2*t)"))
print(get_first_function_name(f))
result = extract_nonpoly_coefficients(f, exp)
print(result)
assert result == None

-2.0*x + 1.5*sin(2.0*x + 2) + 3

False
{a2_: 2.00000000000000, a3_: 2, a1_: 1.50000000000000, a4_: 3/x, a5_: -2.0*x}
False
{a3_: 2.0*x, a2_: 2/t, a1_: 1.50000000000000, a4_: 3/t, a5_: -2.0*x}
False
{a3_: 2.0*x, a2_: 2/t, a1_: 1.50000000000000, a5_: 3 - 2.0*x}
None
True
{a2_: 1, a3_: 0, a1_: 4.40000000000000, a4_: 0, a5_: 0}
[4.4, 1.0, 0.0, 0.0, 0.0]
sin
True
{a2_: 4.00000000000000, a3_: 0, a1_: -3.00000000000000, a4_: 0, a5_: 0}
[-3.0, 4.0, 0.0, 0.0, 0.0]
sin
True
{a2_: 4.00000000000000, a3_: 2.30000000000000, a1_: -3.00000000000000, a4_: 0, a5_: 0}
[-3.0, 4.0, 2.3, 0.0, 0.0]
sin
True
{a4_: 2.00000000000000, a5_: 3.00000000000000, a2_: 2.90000000000000, a3_: 2, a1_: -1.50000000000000}
[-1.5, 2.9, 2.0, 2.0, 3.0]
sin
False
{a4_: 3/t, a2_: 1, a3_: 2, a1_: -1.50000000000000, a5_: -2.0*t}
False
{a2_: 1, a3_: 2, a1_: -1.50000000000000, a5_: 3 - 2.0*t}
None


AssertionError: 

In [3]:
import sympy as sp
from sympy.core.function import AppliedUndef, Function
from utils.utils import *

FUNCTIONS = ["sin", "cos", "exp", "log"]

@timeout(1)
def get_first_function_name(expr):
    # Function to recursively search for the first function in the expression
    if isinstance(expr, Function) and not isinstance(expr, AppliedUndef):
        return expr.func.__name__
    elif expr.args:
        for arg in expr.args:
            name = get_first_function_name(arg)
            if name:
                return name
    return None

# TODO: Only keep functions take the form a*func(b*x+c) + d*x +e

# Return coefficients of something like "exp(-2.0*t+1.5)"
# which should return [-2.0, 0.3]
def get_function_coefficients(f):
    #print("get_function_coefficients:", f)
    function = get_first_function_name(f)
    if function not in FUNCTIONS:
        return None, None
    assert 't' in str(f)
    a,b = 1.0, 0.0
    if len(f.args) > 0:
        expr = f.args[0]  # like "-2.0*t+1.5"
        #print("expr:", expr, expr.args)
        if isinstance(expr, sp.Mul):
            a = float(expr.args[0])
        elif len(expr.args) > 0:
            if expr.args[0].is_constant():
                b = float(expr.args[0])
                sub_expr = expr.args[1]
            else:
                sub_expr = expr
            #print("sub_expr:", sub_expr)
            assert 't' in str(sub_expr)
            if len(sub_expr.args) > 0:
                a = float(sub_expr.args[0])
    #print(a, b)
    return a, b
    

# Return coefficients in a certain order
# For example, for "3-1.5*sin(t+0.3)+2.0*t", returns
# [-1.5, 1.0, 0.3, 2.0, 3.0]
@timeout(5)
def get_nonpoly_coefficients(f):
    function_calls = f.find(Function)
    if len(function_calls) == 0 or len(function_calls) > 1:
        return None
    #print(str(f))
    f_name = None
    coeffs = [1.0, 1.0, 0.0, 0.0, 0.0]
    if len(f.args) == 1:
        a,b = get_function_coefficients(f)
        coeffs[1] = a
        coeffs[2] = b
        if None in coeffs:
            return None
        return coeffs
    if isinstance(f, sp.Mul):
        coeffs[0] = float(f.args[0])
        a,b = get_function_coefficients(f.args[1])
        coeffs[1] = a
        coeffs[2] = b
        if None in coeffs:
            return None
        return coeffs
    for sub_f in f.args:
        if sub_f.is_constant():
            coeffs[4] = float(sub_f)
        elif str(sub_f).endswith("*t"):
            coeffs[3] = float(sub_f.args[0])
        else:
            for n in FUNCTIONS:
                if n in str(sub_f):
                    f_name = n
            if f_name is None:
                raise Exception("Cannot parse function", str(f))
            #print("sub_f", str(sub_f), str(sub_f.args), len(sub_f.args))
            if len(sub_f.args) == 1:
                a,b = get_function_coefficients(sub_f)
            elif sub_f.args[0].is_constant():
                coeffs[0] = float(sub_f.args[0])
                a,b = get_function_coefficients(sub_f.args[1])
            else:
                a,b = get_function_coefficients(sub_f.args[0])
            coeffs[1] = a
            coeffs[2] = b
    if None in coeffs:
        return None
    return coeffs



# expr = f.args[2].args[1].args[0]
# print(expr.args[0])
# sub_expr = expr.args[1]
# sub_expr.args
                        
f = sp.simplify(sp.sympify("4.4*sin(t)"))
print(get_first_function_name(f))
print(get_nonpoly_coefficients(f))
assert get_nonpoly_coefficients(f) == [4.4, 1.0, 0.0, 0.0, 0.0]

f = sp.simplify(sp.sympify('-3.0*sin(4.0*t)'))
print(get_first_function_name(f))
print(get_nonpoly_coefficients(f))
assert get_nonpoly_coefficients(f) == [-3.0, 4.0, 0.0, 0.0, 0.0]

f = sp.simplify(sp.sympify('-3.0*sin(4.0*t+2.3)'))
print(get_first_function_name(f))
print(get_nonpoly_coefficients(f))
assert get_nonpoly_coefficients(f) == [-3.0, 4.0, 2.3, 0.0, 0.0]
    
f = sp.simplify(sp.sympify("3-1.5*sin(2.9*t+2)+2.0*t"))
print(get_first_function_name(f))
assert get_nonpoly_coefficients(f) == [-1.5, 2.9000000000000, 2.0, 2.0, 3.0]
print(get_nonpoly_coefficients(f))

f = sp.simplify(sp.sympify("3-1.5*sin(t+2)-2.0*t"))
print(get_first_function_name(f))
assert get_nonpoly_coefficients(f) == [-1.5, 1.0, 2.0, -2.0, 3.0]
print(get_nonpoly_coefficients(f))

f = sp.simplify(sp.sympify("exp(t)-2.0*t"))
print(get_first_function_name(f))
assert get_nonpoly_coefficients(f) == [1.0, 1.0, 0.0, -2.0, 0.0]
print(get_nonpoly_coefficients(f))

f = sp.simplify(sp.sympify("sin(2.3*t)-3"))
print(get_first_function_name(f))
print(get_nonpoly_coefficients(f))
assert get_nonpoly_coefficients(f) == [1.0, 2.3, 0.0, 0.0, -3.0]

f = sp.simplify(sp.sympify("sin(t)"))
print(get_first_function_name(f))
print(get_nonpoly_coefficients(f))
assert get_nonpoly_coefficients(f) == [1.0, 1.0, 0.0, 0.0, 0.0]

f = sp.simplify(sp.sympify("cos(2*t)"))
print(get_first_function_name(f))
assert get_nonpoly_coefficients(f) == [1.0, 2.0, 0.0, 0.0, 0.0]
print(get_nonpoly_coefficients(f))

f = sp.simplify(sp.sympify("t*exp(2*t)+3"))
print(get_first_function_name(f))
print(get_nonpoly_coefficients(f))
assert get_nonpoly_coefficients(f) == None

f = sp.simplify(sp.sympify("2*t*exp(2*t)"))
print(get_first_function_name(f))
print(get_nonpoly_coefficients(f))
assert get_nonpoly_coefficients(f) == None

f = sp.simplify(sp.sympify("-3.0*sin(4.0*t) + 2.1*t + sin(2.0*t)"))
print(get_first_function_name(f))
print(get_nonpoly_coefficients(f))
assert get_nonpoly_coefficients(f) == None

#f = sp.simplify(sp.sympify("sin(2.3*t)"))
#print(get_function_coefficients(f))

# sub_expr = sp.sympify('t')
# sub_expr.args

sin
[4.4, 1.0, 0.0, 0.0, 0.0]
sin
[-3.0, 4.0, 0.0, 0.0, 0.0]
sin
[-3.0, 4.0, 2.3, 0.0, 0.0]
sin
[-1.5, 2.9, 2.0, 2.0, 3.0]
sin
[-1.5, 1.0, 2.0, -2.0, 3.0]
exp
[1.0, 1.0, 0.0, -2.0, 0.0]
sin
[1.0, 2.3, 0.0, 0.0, -3.0]
sin
[1.0, 1.0, 0.0, 0.0, 0.0]
cos
[1.0, 2.0, 0.0, 0.0, 0.0]
exp
None
exp
None
sin
None


In [4]:
import numpy as np
import sympy as sp
from sympy import sympify, lambdify, symbols, integrate, Interval, Symbol, I, S, oo, plot
from IPython.display import display

# Given an expr f (of variable t), returns its integral, together with t's and y's for regression
def integrate_expr(f, min_x=-4.0, max_x=4.0, increment=0.001, verbose=False):
    if verbose:
        print("Running integration on")
        display(f)
    # Compute integration
    x, t = symbols(['x','t'])
    fi = integrate(f, (t, 0, x))
    if verbose:
        display(fi)
        #plot(fi, (t, min_x, max_x))
    # Generate data for symbolic regression
    fi = fi.subs({x:t})
    fl = lambdify((t), fi, "numpy")
    ts = np.arange(min_x, max_x, increment)
    ys = fl(ts)
    return fi, ts, ys
    
integrate_expr(sympify("2*exp(3*t)+2.5"), verbose=True)

Running integration on


2*exp(3*t) + 2.5

2.5*x + 0.666666666666667*exp(3*x) - 0.666666666666667

(2.5*t + 0.666666666666667*exp(3*t) - 0.666666666666667,
 array([-4.   , -3.999, -3.998, ...,  3.997,  3.998,  3.999]),
 array([-1.06666626e+01, -1.06641626e+01, -1.06616625e+01, ...,
         1.07540373e+05,  1.07863453e+05,  1.08187503e+05]))

In [5]:
from utils.utils import *

def round_expr(expr, num_digits=2):
    return expr.xreplace(Transform(lambda x: x.round(num_digits), lambda x: isinstance(x, sp.Float)))

# Run symbolic regression on given data
# Returns: (raw regressed expr, rounded expr, model refined expr)
@timeout(15)
def symbolic_regress(sr_model, xs, ys, generate_refinement=False, verbose=False):
    if verbose:
        print("Running Symbolic Regression...")
    ##Example of data
    xs = np.reshape(xs, (len(xs),1))
    ys = np.reshape(ys, (len(ys),1))
    sr_model.fit(xs,ys)
    #
    replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/"}
    model_str = sr_model.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
    for op,replace_op in replace_ops.items():
        model_str = model_str.replace(op,replace_op)
    #
    raw_expr = sp.parse_expr(model_str)
    x_0, t = symbols(['x_0', 't'])
    raw_expr = raw_expr.subs(x_0, t)
    if verbose:
        display(raw_expr)
    #
    expr = sp.expand(raw_expr)
    rounded_expr = round_expr(expr)
    if verbose:
        display(rounded_expr)
    #
    # Encode some input text
    if generate_refinement:
        prompt = str(rounded_expr)
        input_ids = tokenizer.encode(prompt, return_tensors='pt')
        #
        # Generate text
        output = model.generate(input_ids, max_length=50, num_return_sequences=1, temperature=0.1)
        #
        # Decode and print the output
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_expr = sympify(generated_text)
        if verbose:
            display(generated_expr)
        #
    else:
        generated_expr = None
    return raw_expr, rounded_expr, generated_expr


fi, ts, ys = integrate_expr(sympify("1.5*sin(-2*t)+1"), verbose=False)
rounded_fi = round_expr(fi)
display(rounded_fi)
raw_expr, rounded_expr, generated_expr = symbolic_regress(est, ts, ys, verbose=False)
display(raw_expr)
display(rounded_expr)

print("Diff1:")
display(rounded_fi-rounded_expr)
# print("Diff2:")
# display(rounded_fi-generated_expr)

t + 0.75*cos(2*t) - 0.75

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')


1.06839146228604*t + (0.42522288721005094*t - 1.6589687209136874)*(-9.936462971084912e-7*sin(39883.287092315097*(1 - 0.0012093902057278237*t)**2 + 0.48904091450850684) - 0.17144524227828595) - 0.7543060932102238*cos(1.994857823498501*t + 3.1223918634934217) - 1.052879826961122

1.0*t - 0.75*cos(1.99*t + 3.12) - 0.77

Diff1:


0.75*cos(2*t) + 0.75*cos(1.99*t + 3.12) + 0.02

In [6]:
import random

template = "c_0*f_0(c_1*t)+c_2"

functions = ['sin', 'cos', 'exp']

def random_constant():
    if random.random() < 0.33:
        return 1
    while True:
        if random.random() < 0.5:
            val = random.randint(-5, 5)
        else:
            val = round(random.uniform(-5.0, 5.0), 1)
        return val

def generate_non_poly_expr():
    func = random.choice(functions)
    while True:
        c_0 = random_constant()
        if c_0 != 0:
            break
    c_1 = random_constant()
    if random.random() < 0.33:
        c_2 = random_constant()
    else:
        c_2 = 0
    expr = template.replace('f_0', str(func)).replace('c_0', str(c_0)).replace('c_1', str(c_1)).replace('c_2', str(c_2))
    return sp.simplify(sympify(expr))

for i in range(10):
    display(generate_non_poly_expr())

exp(t)

-5.0*sin(t) - 4.0

5 - sin(3*t)

4.0 - 3.1*cos(t)

-3*exp(t)

-sin(1.2*t)

-0.5*exp(1.1*t)

cos(5*t) + 2.2

2

cos(3*t) - 4

In [44]:
# exprs = []
# for i in range(200000):
#     if i % 1000 == 0:
#         print(i, "generated")
#     exprs.append(generate_non_poly_expr())

# print(len(exprs))

0 generated
1000 generated
2000 generated
3000 generated
4000 generated
5000 generated
6000 generated
7000 generated
8000 generated
9000 generated
10000 generated
11000 generated
12000 generated
13000 generated
14000 generated
15000 generated
16000 generated
17000 generated
18000 generated
19000 generated
20000 generated
21000 generated
22000 generated
23000 generated
24000 generated
25000 generated
26000 generated
27000 generated
28000 generated
29000 generated
30000 generated
31000 generated
32000 generated
33000 generated
34000 generated
35000 generated
36000 generated
37000 generated
38000 generated
39000 generated
40000 generated
41000 generated
42000 generated
43000 generated
44000 generated
45000 generated
46000 generated
47000 generated
48000 generated
49000 generated
50000 generated
51000 generated
52000 generated
53000 generated
54000 generated
55000 generated
56000 generated
57000 generated
58000 generated
59000 generated
60000 generated
61000 generated
62000 generated
63000

In [ ]:
# # Run symbolic regression on each case
# import json

# seen_exprs = set()

# # fin = open("../datasets/parametric_equations_randomized_nonpoly_integral_results.json", "r")
# # lines = fin.readlines()
# # for line in lines:
# #     result = json.loads(line)
# #     expr = result["original"]
# #     seen_exprs.add(expr)
# # fin.close()
# # print(f"{len(seen_exprs)} exprs loaded")

# fout = open("../datasets/parametric_equations_randomized_nonpoly_integral_results_corrected.json", "a")
# num_seen = 0
# num_seen_changed = False

# for f in exprs:
#     if str(f) in seen_exprs:
#         num_seen+=1
#         num_seen_changed = True
#         continue
#     else:
#         num_seen_changed = False
#     if num_seen_changed:
#         print(f"{num_seen} exprs ignored")
#     #print("Original expr and its integral:")
#     #display(f)
#     #print(f)
#     try:
#         fi, xs, ys = integrate_expr(f, verbose=False)
#         x, t = symbols(['x','t'])
#         fi = fi.subs({x:t})
#         rounded_fi = round_expr(fi)
#         #display(rounded_fi)
#         raw_expr, rounded_expr, generated_expr = symbolic_regress(est, xs, ys, generate_refinement=False, verbose=False)
# #         print("Generated expr:")
# #         display(generated_expr)
#         results = {"original":str(f),
#                    "integral":str(fi),
#                    "rounded_integral":str(rounded_fi),
#                    "regressed":str(raw_expr),
#                    "rounded_regressed":str(rounded_expr),
#                    #"generated_regressed":str(generated_expr),
#                    "diff_rounded": str(rounded_fi-rounded_expr),
#                    #"diff_generated": str(rounded_fi-generated_expr)
#                   }
#         fout.write(json.dumps(results))
#         fout.write('\n')
#         fout.flush()
#     except Exception as e:
#         print("Failed to run symbolic regression")
#         print(e)
#         continue
#     #     print("Diff1:")
#     #     display(rounded_fi-rounded_expr)
#     #     print("Diff2:")
#     #     display(rounded_fi-generated_expr)

# fout.close()

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('gra

Failed to run symbolic regression
Timer expired


/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('gra

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('gra

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('gra

...

Failed to run symbolic regression
Invalid comparison of non-real zoo


/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('gra

Failed to run symbolic regression
Timer expired
Failed to run symbolic regression
object of type 'int' has no len()


/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('gra

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('gra

In [7]:
# Check the accuracy of symbolic regression

import json
from sympy import evalf, N
from utils.utils import *


# Check if f1 and f2 are almost equal.
# Note: Relative error is defined based on f1. Please use the original expression as f1.
def almost_equal(f1, f2, max_abs_error=0.011, max_relative_error=0.011, verbose=False):
    expr = f1-f2
    coeff_pairs = None
    try:
        coeff_pairs = get_coefficients_and_exponents(expr)
    except:
        print("Cannot get_coefficients_and_exponents")
        print(str(expr))
    if coeff_pairs is None:
        constants = get_all_constants(expr)
    else:
        constants = [p[0] for p in coeff_pairs]
    # Check if all diffs are within max_abs_error
    violators = [c for c in constants if c == sp.nan or abs(c) > max_abs_error]
    if verbose:
        print("Violating constants:", violators)
    if len(violators) == 0:
        return True
    # Check if all violating diffs are within max_relative_error
    try:
        coeffs1 = get_polynomial_coeffs(f1)
        coeffs2 = get_polynomial_coeffs(f2)
    except:
        return False
    for i in range(len(coeffs1)):
        if abs(coeffs1[i] - coeffs2[i]) > max_abs_error and \
           abs(coeffs1[i] - coeffs2[i]) > max_relative_error*abs(coeffs1[i]):
            return False
    return True
    

# fin = open("datasets/parametric_equations_polynomial_integral_results.json", "r")
# lines = fin.readlines()

# num_total, qualified_rounded, qualified_generated = 0, 0, 0
# for line in lines:
#     result = json.loads(line)
#     if "diff_rounded" not in result or "diff_generated" not in result:
#         continue
#     rounded_integral = N(sympify(result["rounded_integral"]))
#     rounded_regressed = N(sympify(result["rounded_regressed"]))
# #     display(rounded_integral)
# #     display(rounded_regressed)
#     try:
#         rounded_regressed = filter_non_polynomial(rounded_regressed)
#     except:
#         print("Cannot filter non-polynomials on", str(rounded_regressed))
#     #generated_regressed = N(sympify(result["generated_regressed"]))
#     diff_rounded = rounded_integral - rounded_regressed
#     #diff_generated = sympify(result["diff_generated"])
#     num_total += 1
#     if almost_equal(rounded_integral, rounded_regressed, verbose=False):
#         qualified_rounded += 1
#     else:
#         display(rounded_integral)
#         display(rounded_regressed)
#         print(rounded_regressed)
#         display(diff_rounded)
# #     if is_close_to_zero(diff_generated, True):
# #         qualified_generated += 1
    
# fin.close()

# print(num_total, qualified_rounded, qualified_generated)

In [8]:
# Generate data to infer the rules for integral

import pickle
import json
import numpy as np
import sympy as sp
from sympy import sympify, lambdify, symbols, integrate, Interval, Symbol, I, S, oo, plot, evalf, N
from sympy.core.function import AppliedUndef, UndefinedFunction
from IPython.display import display
from utils.utils import *

MAX_POWER = 6
MAX_AVG_DIFF = 0.01

def remove_constants(f):
    t = Symbol('t')
    return f.as_independent(t)[1]

def round_constants_to_near_integer(expr):
    constants = get_all_constants(expr)
    replacements = {}
    for constant in constants:
        s = str(constant)
        if (".99" in s) or (".98" in s) or (".01" in s) or (".02" in s):
            rounded = constant.round(0)
            replacements[constant] = rounded
    return expr.subs(replacements)

def get_functions(expr):
    functions = list()
    for atom in expr.atoms(AppliedUndef, UndefinedFunction):
        functions.append(atom.func)
    for atom in expr.atoms(sp.sin, sp.cos, sp.exp):  # Add any other functions you're interested in
        functions.append(atom.func)
    return functions

@timeout(5)
# result is a json dict parsed from the line
def parse_row(result):
    try:
        original = round_all_floats(N(sympify(result["original"])))
        original_integral = round_all_floats(N(sympify(result["rounded_integral"])))
        integral = round_constants_to_near_integer(
            #remove_constants(
            round_all_floats(N(sympify(result["rounded_regressed"])))
            #)
        )
        if len(get_functions(integral)) > 1:
            #print("Skipping", result["rounded_regressed"])
            return None, None
        #original = filter_non_polynomial(original)
        #integral = filter_non_polynomial(integral)
        #original_integral = integrate(original, t)
        avg_diff = get_avg_diff(original_integral, integral, t, min_t=0.0, max_t=4.0, increment=0.02)
        if avg_diff > MAX_AVG_DIFF or len(original_integral.args) != len(integral.args):
            #print("Skipping. Diff=", avg_diff)
            #display(original)
            #display(original_integral)
            #display(integral)
            return None, None
    except Exception as e:
        print(e)
        print("Cannot process", str(integral))
        return None, None
    return original, integral

get_functions(sp.sympify("-0.73*t*sin(0.43*t) - 4.63*t + 0.38*sin(0.43*t)"))

# fin = open("../datasets/parametric_equations_randomized_nonpoly_integral_results_corrected.json", "r")
# lines = fin.readlines()
# fin.close()

# originals = []
# integrals = []
# t = Symbol('t')

# for i in range(len(lines)):
#     if i % 100 == 0:
#         print(i, "rows processed")
#     line = lines[i]
#     result = json.loads(line)
#     if "rounded_regressed" not in result:
#         continue
#     #print(line)
#     try:
#         original, integral = parse_row(result)
#     except:
#         print("Cannot process", str(integral))
#         continue
#     if integral is None: continue
#     originals.append(original)
#     integrals.append(integral)
#     if len(originals) % 100 == 0:
#         print(len(originals), "valid cases loaded")
    
# fin.close()

# fout = open("../datasets/parametric_equations_randomized_nonpoly_integral_results_corrected.pkl", "wb")
# pickle.dump(originals, fout)
# pickle.dump(integrals, fout)
# fout.close()

[sin]

In [16]:
expr = sp.sympify("-0.73*t*sin(0.43*t) - 4.63*t + 0.38*sin(0.43*t)")
expr.atoms(AppliedUndef, UndefinedFunction)

set()

In [11]:
import random

for i in range(100):
    idx = random.randint(0, 40000)
    print(idx)
    display(originals[idx])
    display(integrals[idx])

12828


exp(-3.4*t)

0.29 - 0.29*exp(-3.4*t)

7830


-5.0*cos(t)

...

exp(2*t)

0.5*exp(2.0*t) - 0.5

13533


-sin(1.5*t)

-1.34*sin(0.75*t)**2

In [10]:
import pickle

fin = open("../datasets/parametric_equations_randomized_nonpoly_integral_results_corrected.pkl", "rb")
originals = pickle.load(fin)
integrals = pickle.load(fin)
fin.close()

wrong_idx = -1
for i in range(len(integrals)):
    if str(integrals[i]) == "0.48*cos(2.10009765625*t + 4.2673938390114e-118662491247) - 0.48":
        print(i)
        wrong_idx = i
        break

print(wrong_idx)
print(integrals[wrong_idx])
del originals[wrong_idx]
del integrals[wrong_idx]

-1
1.25 - 1.25*exp(-4.0*t)


In [11]:
# Generate data for regression to infer the rules for integral
# The data is in the following format:
# For each function (sin, cos, exp), we keep the 5 coefficients for the original
# and 5 coeffients for the integral, in two separate lists of lists (can be convered into NP arrays)

from sympy import evalf, N
from utils.utils import *
import pickle

# fin = open("../datasets/parametric_equations_randomized_nonpoly_integral_results_corrected.pkl", "rb")
# originals = pickle.load(fin)
# integrals = pickle.load(fin)
# fin.close()

#originals = [sp.sympify("-sin(2.1*t) ")] 
#integrals = [sp.sympify("0.48*cos(2.10009765625*t + 4.2673938390114e-118662491247) - 0.48")]

print(len(originals), "records loaded")

NUM_COEFFS = 5

FUNCTIONS = ["sin", "cos", "exp", "log"]

data_series = {}
for f in FUNCTIONS:
    # Original coeffs, integral coeffs, integral func names, original, integral
    data_series[f] = (list(), list(), list(), list(), list()) 
t = Symbol('t')

for i in range(len(originals)):
    #if i > 11700:
    #print("Processing", originals[i], integrals[i])
    orig_f_name = get_first_function_name(originals[i])
    intg_f_name = get_first_function_name(integrals[i])
    if orig_f_name not in FUNCTIONS or intg_f_name not in FUNCTIONS:
        continue
    #print(originals[i], integrals[i])
    try:
        orig_coeffs = get_nonpoly_coefficients(originals[i])
        intg_coeffs = get_nonpoly_coefficients(sp.sympify(str(integrals[i])))
        max_value = max([abs(x) for x in orig_coeffs])
        max_diff = 0
        for j in range(len(orig_coeffs)):
            max_diff = max(max_diff, abs(orig_coeffs[j] - intg_coeffs[j]))
        if max_diff > max_value * 1.5:
            print("Abnormal values:", orig_coeffs, intg_coeffs)
            continue
    except Exception as e:
        print(originals[i], integrals[i])
        print(e)
        continue
    data_series[orig_f_name][0].append(orig_coeffs)
    data_series[orig_f_name][1].append(intg_coeffs)
    data_series[orig_f_name][2].append(intg_f_name)
    data_series[orig_f_name][3].append(str(originals[i]))
    data_series[orig_f_name][4].append(str(integrals[i]))
    if i % 100 == 0:
        print(i, "processed")


41622 records loaded
0 processed
Abnormal values: [0.7998046875, 4.0, 0.0, 0.0, 0.0] [0.2, 4.0, -64.4, 0.0, 0.0]
Abnormal values: [4.7001953125, 5.0, 0.0, 0.0, 0.0] [-0.94, 5.0, 15.71, 0.0, 0.0]
Abnormal values: [2.0, 4.7998046875, 0.0, 0.0, 0.0] [-0.42, 4.8, -135.09, 0.0, 0.0]
Abnormal values: [2.0, 3.0, 0.0, 0.0, 0.0] [-0.67, 3.0, 51.84, 0.0, 0.0]
-4.4*sin(t) -8.78*sin(0.5*t)**2 + 0.03*sin(0.5*t)
Cannot convert expression to float
0.8*sin(5*t) 0.32*sin(2.5*t)**2
Cannot convert expression to float
3.0*sin(t) -0.04*t*sin(0.5*t) + 6.09*sin(0.5*t)**2
Cannot convert expression to float
Abnormal values: [2.0, 3.39990234375, 0.0, 0.0, 0.0] [0.59, 3.4, -7.85, 0.0, 0.59]
-5.0*sin(2.0*t) -5.0*sin(1.0*t)**2
Cannot convert expression to float
sin(t) + 4.0 0.09*t**2 + 0.51*t*sin(0.76*t - 0.08) + 4.0*t
('Cannot parse function', '0.09*t**2 + 0.51*t*sin(0.76*t - 0.08) + 4.0*t')
Abnormal values: [1.0, -1.0, 0.0, 0.0, 0.0] [-1.0, -1.0, 0.0, 0.0, 1.0]
2.0*sin(t) 4.0*sin(0.5*t)**2
Cannot convert express

4.0*sin(t) 8.04*sin(0.5*t)**2
Cannot convert expression to float
-2.0*sin(t) -4.03*sin(0.5*t)**2
Cannot convert expression to float
Abnormal values: [1.0, 5.0, 0.0, 0.0, 1.0] [-0.2, 5.0, 711.57, 1.0, 0.2]
-3.0*sin(t) -6.0*sin(0.5*t)**2
Cannot convert expression to float
2.4*sin(2*t) 2.4*sin(1.0*t)**2
Cannot convert expression to float
Abnormal values: [2.2998046875, -1.0, 0.0, 0.0, 0.0] [-2.3, -1.0, 0.0, 0.0, 2.3]
Abnormal values: [2.0, 1.0, 0.0, 0.0, 0.0] [2.0, 1.0, 23.57, 0.0, 0.0]
sin(t) 2.0*sin(0.5*t)**2
Cannot convert expression to float
-5.0*sin(t) -10.0*sin(0.5*t)**2
Cannot convert expression to float
5.0*sin(t) 10.03*sin(0.5*t)**2
Cannot convert expression to float
-sin(t) -2.0*sin(0.5*t)**2
Cannot convert expression to float
sin(t) 2.0*sin(0.5*t)**2
Cannot convert expression to float
Abnormal values: [1.0, -1.0, 0.0, 0.0, 0.0] [-1.0, -1.0, 0.0, 0.0, 1.0]
5.0*sin(4.9*t) 2.04*sin(2.45*t)**2
Cannot convert expression to float
-2.7*sin(4*t) -1.35*cos(2.0*t - 7.85)**2
Cannot conver

-sin(0.5*t) -0.06*t**2*cos(0.43*t) - 0.19*t**2
('Cannot parse function', '-0.06*t**2*cos(0.43*t) - 0.19*t**2')
Abnormal values: [1.0, -1.0, 0.0, 0.0, 0.0] [-1.0, -1.0, 0.0, 0.0, 1.0]
Abnormal values: [-2.0, -1.800048828125, 0.0, 0.0, 0.0] [1.11, -1.8, 0.0, 0.0, -1.11]
Abnormal values: [-2.60009765625, -1.0, 0.0, 0.0, 0.0] [2.6, -1.0, 0.0, 0.0, -2.6]
-5.0*sin(t) -9.9*sin(0.5*t)**2 - 0.e-2*sin(0.5*t)
Cannot convert expression to float
Abnormal values: [5.0, 3.0, 0.0, 0.0, 0.0] [1.67, 3.0, -51.84, 0.0, 0.0]
Abnormal values: [4.099609375, 3.0, 0.0, 0.0, 0.0] [1.37, 3.0, -416.25, 0.0, 1.37]
sin(t) 2.0*sin(0.5*t)**2
Cannot convert expression to float
-5.0*sin(1.3*t) -7.69*sin(0.65*t)**2
Cannot convert expression to float
Abnormal values: [-2.7998046875, -2.0, 0.0, 0.0, 0.0] [1.4, -2.0, 0.0, 0.0, -1.4]
Abnormal values: [2.2998046875, 4.0, 0.0, 0.0, -0.7001953125] [-0.57, 4.0, -55.0, -0.7, 0.0]
-4.0*sin(5*t) -1.6*cos(2.5*t - 36.13)**2
Cannot convert expression to float
-3.0*sin(1.8*t) -3.33*si

1200 processed
-5.0*sin(2.0*t) -5.0*sin(1.0*t)**2
Cannot convert expression to float
Abnormal values: [1.0, 2.39990234375, 0.0, 0.0, -2.5] [0.41, 2.42, -26.65, -2.5, 0.4]
4.0*sin(2*t) 4.0*sin(1.0*t)**2
Cannot convert expression to float
3.0*sin(2.4*t) 2.5*sin(1.2*t)**2
Cannot convert expression to float
3.0*sin(3*t) 2.0*sin(1.5*t)**2
Cannot convert expression to float
2.9*sin(4*t) 1.45*sin(2.0*t)**2
Cannot convert expression to float
-sin(1.4*t) -1.43*sin(0.7*t)**2
Cannot convert expression to float
sin(4*t) 0.5*sin(2.0*t)**2
Cannot convert expression to float
Abnormal values: [2.0, 2.7998046875, 0.0, 0.0, 0.0] [0.71, 2.8, -14.14, 0.0, 0.71]
Abnormal values: [2.60009765625, 3.0, 0.0, 0.0, 0.0] [0.87, 3.0, -12.57, 0.0, 0.0]
-0.8*sin(2*t) -0.8*sin(1.0*t)**2
Cannot convert expression to float
sin(t) 2.0*sin(0.5*t)**2
Cannot convert expression to float
3.4*sin(2.9*t) 2.34*sin(1.45*t)**2
Cannot convert expression to float
4.0*sin(3.9*t) 2.05*sin(1.95*t)**2
Cannot convert expression to float

Abnormal values: [-3.5, 3.0, 0.0, 0.0, 0.0] [1.17, 3.0, 76.97, 0.0, 0.0]
-2.6*sin(t) -5.2*sin(0.5*t)**2
Cannot convert expression to float
5.0*sin(4*t) 2.5*sin(2.0*t)**2
Cannot convert expression to float
4.0*sin(t) 8.0*sin(0.5*t)**2
Cannot convert expression to float
-sin(5*t) 0.4*cos(2.5*t - 3.14)**2 - 0.4
Cannot convert expression to float
3.0*sin(4.8*t) 1.25*sin(2.4*t)**2
Cannot convert expression to float
3.1*sin(2.9*t) 2.13*sin(1.45*t)**2
Cannot convert expression to float
cos(t) - 5.0 0.5*t*cos(0.74*t) - 4.63*t
'NoneType' object is not subscriptable
sin(3*t) 0.66*sin(1.5*t)**2
Cannot convert expression to float
-5.0*sin(5*t) -2.0*sin(2.5*t)**2
Cannot convert expression to float
1600 processed
Abnormal values: [4.7998046875, 5.0, 0.0, 0.0, 0.0] [0.96, 5.0, 50.27, 0.0, 0.0]
Abnormal values: [-3.0, 5.0, 0.0, 0.0, 0.0] [0.6, 5.0, -17.28, 0.0, 0.0]
1.0 - 3.0*cos(0.8*t) -1.77*t*cos(0.59*t) - 0.2*t
'NoneType' object is not subscriptable
2.0*exp(0.4*t) 1.6*t*exp(0.26*t) + 0.39*t
'NoneTy

...

39900 processed
3.0*sin(3.5*t) 1.71*sin(1.75*t)**2
Cannot convert expression to float
Abnormal values: [-2.0, 2.0, 0.0, 0.0, 0.0] [1.0, 2.0, -80.11, 0.0, 0.0]
-0.6*sin(2.4*t) -0.5*sin(1.2*t)**2
Cannot convert expression to float
Abnormal values: [2.7998046875, -1.699951171875, 0.0, 0.0, 0.0] [-1.65, -1.7, 0.0, 0.0, 1.65]
Abnormal values: [-2.5, -1.39990234375, 0.0, 0.0, 0.0] [1.79, -1.4, 0.0, 0.0, -1.79]
-sin(3.2*t) -0.62*sin(1.6*t)**2
Cannot convert expression to float
Abnormal values: [-3.0, 2.89990234375, 0.0, 0.0, 1.0] [-1.03, 2.9, -3696.09, 1.0, 0.0]
4.5*sin(3.0*t) 3.0*sin(1.5*t)**2
Cannot convert expression to float
4.9*sin(4*t) 2.45*sin(2.0*t)**2
Cannot convert expression to float
Abnormal values: [-4.0, 2.0, 0.0, 0.0, 0.0] [-2.0, 2.0, -301.59, 0.0, 0.0]
-sin(2*t) -1.0*sin(1.0*t)**2
Cannot convert expression to float
-sin(4*t) -0.5*sin(2.0*t)**2
Cannot convert expression to float
sin(2*t) 1.0*sin(1.0*t)**2
Cannot convert expression to float
Abnormal values: [4.0, 4.0, 0.0, 0.0, 

Abnormal values: [1.0, -1.0, 0.0, 0.0, 0.0] [-1.0, -1.0, 0.0, 0.0, 1.0]
sin(3.2*t) 0.62*sin(1.6*t)**2
Cannot convert expression to float
Abnormal values: [-2.60009765625, -1.0, 0.0, 0.0, 0.0] [2.6, -1.0, 0.0, 0.0, -2.6]
5.0*sin(3*t) 3.33*sin(1.5*t)**2
Cannot convert expression to float
-sin(t) -2.0*sin(0.5*t)**2
Cannot convert expression to float
sin(t) 2.0*sin(0.5*t)**2
Cannot convert expression to float
Abnormal values: [-4.0, 4.400390625, 0.0, 0.0, 0.0] [0.91, 4.4, -34.56, 0.0, 0.0]
3.5*sin(t) 7.0*sin(0.5*t)**2
Cannot convert expression to float
-4.7*sin(t) -9.38*sin(0.5*t)**2 + 0.03*sin(0.5*t)
Cannot convert expression to float
40300 processed
5.0*sin(3.6*t) 2.78*sin(1.8*t)**2
Cannot convert expression to float
3.2*sin(3.6*t) 1.78*sin(1.8*t)**2
Cannot convert expression to float
Abnormal values: [3.0, 5.0, 0.0, 0.0, 3.60009765625] [-0.6, 5.0, -11.05, 3.59, 0.56]
4.0*sin(t) 8.0*sin(0.5*t)**2
Cannot convert expression to float
-5.0*sin(5*t) -2.0*sin(2.5*t)**2
Cannot convert expressio

sin(0.4*t) 0.07*t**2*cos(0.27*t) + 0.13*t**2
('Cannot parse function', '0.07*t**2*cos(0.27*t) + 0.13*t**2')
5.0*sin(t) 10.04*sin(0.5*t)**2 + 0.04*sin(0.5*t)
Cannot convert expression to float
-sin(2*t) -1.0*sin(1.0*t)**2
Cannot convert expression to float
sin(1.1*t) 1.81*sin(0.55*t)**2
Cannot convert expression to float
Abnormal values: [4.0, 1.60009765625, 0.0, 0.0, 0.0] [-2.5, 1.6, -530.93, 0.0, 0.0]
Abnormal values: [-1.0, 3.39990234375, 0.0, 0.0, 0.0] [-0.29, 3.4, 11.0, 0.0, -0.29]
3.4*sin(3.9*t) 1.74*sin(1.95*t)**2
Cannot convert expression to float
3.6*sin(3.1*t) 2.32*sin(1.55*t)**2
Cannot convert expression to float
sin(t) 2.0*sin(0.5*t)**2
Cannot convert expression to float
4.0*sin(4.8*t) 1.67*sin(2.4*t)**2
Cannot convert expression to float
-sin(5.0*t) -0.4*sin(2.5*t)**2
Cannot convert expression to float
Abnormal values: [1.0, 4.0, 0.0, 0.0, 0.0] [-0.25, 4.0, 14.14, 0.0, 0.25]
Abnormal values: [1.0, 0.599609375, 0.0, 0.0, 1.0] [5.14, 0.38, 0.0, 0.0, 0.0]
Abnormal values: [2.1

-4.0*sin(3.8*t) -2.11*sin(1.9*t)**2
Cannot convert expression to float
2.8 - 2.0*cos(t) -1.21*t*cos(0.73*t) + 2.0*t
'NoneType' object is not subscriptable
5.0*sin(2*t) 5.03*sin(1.0*t)**2
Cannot convert expression to float
Abnormal values: [1.0, -1.199951171875, 0.0, 0.0, 0.0] [-0.83, -1.2, 0.0, 0.0, 0.83]
Abnormal values: [1.0, 5.0, 0.0, 0.0, 0.0] [-0.2, 5.0, -11.0, 0.0, 0.2]
5.0*sin(t) 10.0*sin(0.5*t)**2
Cannot convert expression to float
Abnormal values: [4.0, 5.0, 0.0, 0.0, 0.0] [-0.8, 5.0, 9.42, 0.0, 0.0]
5.0*sin(4*t) 2.5*sin(2.0*t)**2
Cannot convert expression to float
-2.0*sin(3*t) -1.33*sin(1.5*t)**2
Cannot convert expression to float
4.8*sin(5*t) 1.92*sin(2.5*t)**2
Cannot convert expression to float
Abnormal values: [-3.89990234375, 4.2001953125, 0.0, 0.0, 4.099609375] [0.92, 4.21, -56.51, 4.11, -0.89]
sin(t) 2.0*sin(0.5*t)**2
Cannot convert expression to float
-sin(1.4*t) -1.43*sin(0.7*t)**2
Cannot convert expression to float
-2.0*sin(2*t) -2.0*sin(1.0*t)**2
Cannot convert exp

sin(4.3*t) 0.46*sin(2.15*t)**2
Cannot convert expression to float
3.0*sin(t) 6.0*sin(0.5*t)**2
Cannot convert expression to float
-sin(5*t) -0.4*cos(2.5*t + 20.42)**2
Cannot convert expression to float
Abnormal values: [-3.0, -1.0, 0.0, 0.0, 0.0] [3.0, -1.0, 0.0, 0.0, -3.0]
-2.0*sin(t) -4.0*sin(0.5*t)**2
Cannot convert expression to float
sin(1.6*t) 1.25*sin(0.8*t)**2
Cannot convert expression to float
Abnormal values: [4.599609375, 3.2001953125, 0.0, 0.0, 0.0] [-1.44, 3.2, -9.42, 0.0, 0.0]
sin(3*t) 0.66*sin(1.5*t)**2
Cannot convert expression to float
0.3*sin(t) 0.6*sin(0.5*t)**2
Cannot convert expression to float
Abnormal values: [1.0, -1.0, 0.0, 0.0, 0.0] [-1.0, -1.0, 0.0, 0.0, 1.0]
41500 processed
-4.4*sin(t) -8.78*sin(0.5*t)**2 + 0.03*sin(0.5*t)
Cannot convert expression to float
2.9*sin(t) 5.82*sin(0.5*t)**2
Cannot convert expression to float
sin(3*t) 0.66*sin(1.5*t)**2
Cannot convert expression to float
Abnormal values: [-2.0, -1.10009765625, 0.0, 0.0, 0.0] [1.82, -1.1, 0.0, 0.0

In [10]:
for i in range(50):
    f_name = get_first_function_name(originals[i])
    if f_name == 'exp':
        print(i)
        display(originals[i])
        display(integrals[i])
        orig_coeffs = get_nonpoly_coefficients(originals[i])
        intg_coeffs = get_nonpoly_coefficients(sp.sympify(str(integrals[i])))
        print(orig_coeffs)
        print(intg_coeffs)

0


4.5*exp(-4.0*t)

1.13 - 1.13*exp(-4.0*t)

[4.5, -4.0, 0.0, 0.0, 0.0]
[-1.13, -4.0, 0.0, 0.0, 1.13]
1


2.3 + 3.0*exp(-3.1*t)

...

1.0 - 1.0*exp(-1.0*t)

[1.0, -1.0, 0.0, 0.0, 0.0]
[-1.0, -1.0, 0.0, 0.0, 1.0]
47


exp(-4.3*t)

0.23 - 0.23*exp(-4.3*t)

[1.0, -4.2998046875, 0.0, 0.0, 0.0]
[-0.23, -4.3, 0.0, 0.0, 0.23]


In [31]:
f_name = 'sin'

X = np.asarray(data_series[f_name][0])
Y = np.asarray(data_series[f_name][1])[:,0]

X1 = list()
Y1 = list()

num_violation = 0
for i in range(len(X)):
    pred = -X[i][0]/X[i][1]
    if abs(pred - Y[i]) > 1.0:
        print(i, pred, Y[i], X[i])
        original = sp.sympify(data_series[f_name][3][i])
        integral = sp.sympify(data_series[f_name][4][i])
        original_integral = integrate_expr(original)[0]
        print(original)
        print(original_integral)
        print(integral)
        avg_diff = get_avg_diff(original_integral, integral, t, min_t=0.0, max_t=4.0, increment=0.02)
        print("avg_diff:", avg_diff)
        num_violation += 1
    else:
        X1.append(X[i])
        Y1.append(Y[i])

print(num_violation, "of", len(X), "violated")

34 -0.7958349940215225 0.8 [3.89990234 4.90039062 0.         0.         0.        ]
3.9*sin(4.9*t)
0.795918367346939 - 0.795918367346939*cos(4.9*t)
0.8*sin(4.9*t + 4.71) + 0.8
avg_diff: 0.005198865429209447
51 0.9000244140625 -0.9 [-3.60009766  4.          0.          0.          0.        ]
-3.6*sin(4*t)
0.9*cos(4*t) - 0.9
-0.9*sin(4.0*t - 1.57) - 0.9
avg_diff: 0.0004927641644860577
61 -0.75 0.75 [3. 4. 0. 0. 0.]
3.0*sin(4*t)
0.75 - 0.75*cos(4*t)
0.75*sin(4.0*t + 4.71) + 0.75
avg_diff: 0.0014784113531802591
80 -4.5 4.5 [4.5 1.  0.  0.  0. ]
4.5*sin(t)
4.5 - 4.5*cos(t)
4.5*sin(1.0*t - 1.57) + 4.5
avg_diff: 0.00039283169950342714
81 -0.6000162760416666 0.6 [1.80004883 3.         0.         0.         0.        ]
1.8*sin(3.0*t)
0.6 - 0.6*cos(3.0*t)
0.6*cos(3.0*t + 3.14) + 0.6
avg_diff: 0.0009947154448455438
97 -1.1628435157846924 1.16 [5.         4.29980469 0.         0.         4.        ]
5.0*sin(4.3*t) + 4.0
4.0*t - 1.16279069767442*cos(4.3*t) + 1.16279069767442
4.0*t + 1.16*sin(4.3*t

-3.2*sin(3.8*t)
0.842105263157895*cos(3.8*t) - 0.842105263157895
-0.84*sin(3.8*t - 1.57) - 0.84
avg_diff: 0.002420124898595802
1691 -4.5 4.5 [4.5 1.  0.  0.  0. ]
4.5*sin(t)
4.5 - 4.5*cos(t)
4.5*sin(1.0*t - 1.57) + 4.5
avg_diff: 0.00039283169950342714
1702 3.39990234375 -3.4 [-3.39990234  1.          0.          0.          0.        ]
-3.4*sin(t)
3.4*cos(t) - 3.4
-3.4*sin(1.0*t - 1.57) - 3.4
avg_diff: 0.0003928316995034195
1730 1.25 -1.24 [-5.  4.  0.  0.  0.]
-5.0*sin(4*t)
1.25*cos(4*t) - 1.25
-1.24*cos(4.0*t + 3.14) - 1.24
avg_diff: 0.008201913615765553
1753 -0.7000325520833334 0.69 [2.10009766 3.         0.         0.         5.        ]
2.1*sin(3*t) + 5.0
5.0*t - 0.7*cos(3*t) + 0.7
5.0*t + 0.69*sin(3.0*t - 1.52) + 0.75
avg_diff: 0.004661263144820301
1754 0.7999674479166666 -0.8 [-2.39990234  3.          0.          0.         -2.10009766]
-2.4*sin(3*t) - 2.1
-2.1*t + 0.8*cos(3*t) - 0.8
-2.09*t - 0.8*sin(3.0*t - 1.62) - 0.78
avg_diff: 0.008150002481322723
1784 0.6471348556656613 -0

4.5*sin(t)
4.5 - 4.5*cos(t)
4.5*sin(1.0*t - 1.57) + 4.5
avg_diff: 0.00039283169950342714
2993 -0.512833354200576 0.51 [2.         3.89990234 0.         0.         0.        ]
2.0*sin(3.9*t)
0.512820512820513 - 0.512820512820513*cos(3.9*t)
0.51*sin(3.9*t - 1.57) + 0.51
avg_diff: 0.0054020220781968884
3020 1.0 -1.0 [-5.  5.  0.  0.  1.]
1.0 - 5.0*sin(5*t)
1.0*t + 1.0*cos(5*t) - 1.0
1.0*t - 1.0*cos(5.0*t - 3.14) - 1.0
avg_diff: 0.0007154247354469886
3071 -1.125 1.13 [4.5 4.  0.  0.  0. ]
4.5*sin(4*t)
1.125 - 1.125*cos(4*t)
1.13*cos(4.0*t - 3.14) + 1.12
avg_diff: 0.004182673919619245
3074 0.562480927677754 -0.56 [-1.80004883  3.20019531  0.          0.          0.        ]
-1.8*sin(3.2*t)
0.5625*cos(3.2*t) - 0.5625
-0.56*sin(3.2*t + 4.71) - 0.56
avg_diff: 0.0045726765905261075
3076 0.9118196179807554 -0.91 [-3.10009766  3.39990234  0.          0.          0.        ]
-3.1*sin(3.4*t)
0.911764705882353*cos(3.4*t) - 0.911764705882353
-0.91*sin(3.4*t + 4.71) - 0.91
avg_diff: 0.0024396625637685

In [18]:
import scipy

X1 = np.asarray(X1)
Y1 = np.asarray(Y1)

scipy.stats.linregress(X1[:,0]/X1[:,1], Y1)

LinregressResult(slope=-0.9963437865254879, intercept=0.0020837160406581012, rvalue=-0.9983301542350789, pvalue=0.0, stderr=0.000701965634613795, intercept_stderr=0.0006158523005585108)

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import scipy

X = np.asarray(data_series['sin'][0]).astype(float)
Y = np.asarray(data_series['sin'][1]).astype(float)

print(type(X), X.shape)
print(type(Y), Y.shape)
print(X.dtype)
print(Y.dtype)
print(np.any(np.isnan(X)))
print(np.any(np.isfinite(X)))
print(np.any(np.isnan(Y)))
print(np.any(np.isfinite(Y)))

print("Single variables")
for y_idx in range(5):
    print("\ny_idx:", y_idx)
    for x_idx in range(5):
        print("x_idx:", x_idx)
        try:
            print(scipy.stats.linregress(X[:,x_idx], Y[:,y_idx]))
        except Exception as e:
            print(e)
            
print("Combination of pairs of variables")
for y_idx in [4]:
    print("\ny_idx:", y_idx)
    for x_idx in range(5):
        for x_idx2 in range(5):
            print("x_indices:", x_idx, x_idx2)
            try:
                print(scipy.stats.linregress(X[:,x_idx]+X[:,x_idx2], Y[:,y_idx]))
#                 model = LinearRegression()
#                 model.fit(X, Y[:,y_idx])
#                 print(model.coef_)
#                 print(model.intercept_)
#                 print("R-squared:", model.score(X, Y[:,y_idx]))
            except Exception as e:
                print(e)

# model = LinearRegression()

# # Training the model
# model.fit(X, y)

# print(model.coef_)
# print(model.intercept_)

# # Making predictions
# y_pred = model.predict(X)

# # Evaluating the model
# mse = mean_squared_error(y, y_pred)

# r_squared = model.score(X, y)
# print("R-squared:", r_squared)

#np.concatenate([X, y.reshape(-1, 1)], axis=1)

<class 'numpy.ndarray'> (3953, 5)
<class 'numpy.ndarray'> (3953, 5)
float64
float64
False
True
False
True
Single variables

y_idx: 0
x_idx: 0
LinregressResult(slope=-0.1988253645747498, intercept=0.06008232215093372, rvalue=-0.6150900585608452, pvalue=0.0, stderr=0.004054688552259376, intercept_stderr=0.011682326187546166)
x_idx: 1
LinregressResult(slope=-0.12116306049458449, intercept=0.5900157143292355, rvalue=-0.12266924993568454, pvalue=9.99065062549359e-15, stderr=0.015595115644985889, intercept_stderr=0.05894437719460419)
x_idx: 2
Cannot calculate a linear regression if all x values are identical
x_idx: 3
Cannot calculate a linear regression if all x values are identical
x_idx: 4
LinregressResult(slope=-0.022184253140987955, intercept=0.14883550807350518, rvalue=-0.04240836678842264, pvalue=0.007660164006139861, stderr=0.008314743679805846, intercept_stderr=0.014668253191720855)

y_idx: 1
x_idx: 0
LinregressResult(slope=-0.003991502212608123, intercept=3.6576724934595255, rvalue=

In [205]:
x = X[:,1]
y = Y[:,1]
for i in range(len(x)):
    if abs(x[i]-y[i]) > 10:
        print(x[i], y[i])

1.0 20570.82
1.5 56550.17
2.0 40.0
1.0 -24.64
2.2998046875 72.1


In [198]:
Y[:,1][0:100]

array([1.  , 4.  , 1.  , 5.  , 2.  , 1.  , 4.  , 1.  , 1.  , 4.  , 1.  ,
       3.  , 5.  , 1.  , 1.  , 1.  , 1.  , 0.5 , 2.12, 1.  , 1.  , 2.  ,
       1.  , 1.  , 5.  , 0.5 , 4.  , 1.  , 1.  , 5.  , 4.  , 1.  , 1.  ,
       1.  , 5.  , 2.54, 3.  , 1.  , 1.  , 3.  , 1.4 , 2.  , 3.1 , 4.  ,
       2.  , 1.4 , 2.2 , 1.  , 1.6 , 1.  , 1.  , 1.  , 4.  , 1.9 , 3.8 ,
       3.  , 1.  , 1.  , 2.  , 1.  , 1.  , 1.  , 3.  , 3.  , 1.  , 1.  ,
       1.  , 1.6 , 4.  , 1.  , 4.  , 4.  , 4.4 , 1.  , 1.  , 1.  , 1.  ,
       0.2 , 3.6 , 4.  , 2.  , 1.  , 1.  , 1.  , 1.  , 1.3 , 1.  , 4.  ,
       1.  , 1.  , 1.  , 3.  , 1.49, 2.5 , 4.2 , 1.  , 1.  , 2.4 , 3.2 ,
       1.  ])

In [184]:
est.fit(X, y)
replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/"}
model_str = est.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
for op,replace_op in replace_ops.items():
    model_str = model_str.replace(op,replace_op)
print(model_str)
raw_expr = sp.parse_expr(model_str)
display(raw_expr)

/home/mcwave/code/automath/calculus/nonpoly/symbolicregression/model/utils_wrapper.py:107: RuntimeWarning: divide by zero encountered in divide
  a, b = 1/s, -m/s
/home/mcwave/code/automath/calculus/nonpoly/symbolicregression/model/utils_wrapper.py:107: RuntimeWarning: invalid value encountered in divide
  a, b = 1/s, -m/s


(((0.02 * (0.5991459241539399 + (0.4785720053406121 * x_1))) * (90.0 + (-0.1 * (-0.16584114042813328 + (0.36244114502442476 * x_0))))) + (-1.1 + (-0.1 * exp(((-0.9 + (-0.01 * (-0.004224377071759902 + (0.8138051919473679 * x_4)))) + ((-9.0 + (-4.0 * (nan + (inf * x_2)))) + (0.001 + (-3.0 * (nan + (inf * x_3))))))))))


/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


nan